In [1]:
import torch 
from PIL import Image
from torch import nn, save, load
from torch.optim import Adam
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [2]:
torch. cuda. is_available()

True

In [3]:
train = datasets.MNIST(root="data", download=True, train=True, transform=ToTensor())
dataset = DataLoader(train, 32)

100%|████████████████████████████████████████████████████████████████████| 9912422/9912422 [00:18<00:00, 539462.13it/s]


Extracting data\MNIST\raw\train-images-idx3-ubyte.gz to data\MNIST\raw



100%|███████████████████████████████████████████████████████████████████████| 28881/28881 [00:00<00:00, 1538211.50it/s]


Extracting data\MNIST\raw\train-labels-idx1-ubyte.gz to data\MNIST\raw



100%|████████████████████████████████████████████████████████████████████| 1648877/1648877 [00:02<00:00, 644127.87it/s]


Extracting data\MNIST\raw\t10k-images-idx3-ubyte.gz to data\MNIST\raw



100%|█████████████████████████████████████████████████████████████████████████| 4542/4542 [00:00<00:00, 4559724.45it/s]


Extracting data\MNIST\raw\t10k-labels-idx1-ubyte.gz to data\MNIST\raw



In [4]:
# Image Classifier Neural Network
class ImageClassifier(nn.Module): 
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Conv2d(1, 32, (3,3)), 
            nn.ReLU(),
            nn.Conv2d(32, 64, (3,3)), 
            nn.ReLU(),
            nn.Conv2d(64, 64, (3,3)), 
            nn.ReLU(),
            nn.Flatten(), 
            nn.Linear(64*(28-6)*(28-6), 10)  
        )

    def forward(self, x): 
        return self.model(x)

In [5]:
clf = ImageClassifier().to('cuda')
opt = Adam(clf.parameters(), lr=1e-3)
loss_fn = nn.CrossEntropyLoss() 

In [6]:
from tqdm import tqdm

In [7]:
# Training flow 
if __name__ == "__main__": 
    for epoch in tqdm(range(10)): # train for 10 epochs
        for batch in dataset: 
            X,y = batch 
            X, y = X.to('cuda'), y.to('cuda') 
            yhat = clf(X) 
            loss = loss_fn(yhat, y) 

            # Apply backprop 
            opt.zero_grad()
            loss.backward() 
            opt.step() 

        print(f"Epoch:{epoch} loss is {loss.item()}")

 10%|████████▎                                                                          | 1/10 [00:33<05:05, 33.95s/it]

Epoch:0 loss is 0.09445414692163467


 20%|████████████████▌                                                                  | 2/10 [00:39<02:20, 17.53s/it]

Epoch:1 loss is 0.001387382042594254


 30%|████████████████████████▉                                                          | 3/10 [00:46<01:26, 12.31s/it]

...

Epoch:7 loss is 3.188765549566597e-05


 90%|██████████████████████████████████████████████████████████████████████████▋        | 9/10 [01:22<00:06,  6.51s/it]

Epoch:8 loss is 7.480217755073681e-05


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [01:28<00:00,  8.84s/it]

Epoch:9 loss is 1.3696110727323685e-05


In [8]:
 with open('model_state.pt', 'wb') as f: 
        save(clf.state_dict(), f)

In [9]:
with open('model_state.pt', 'rb') as f: 
        clf.load_state_dict(load(f))  

In [12]:
img = Image.open('img_1.jpg') 
img_tensor = ToTensor()(img).unsqueeze(0).to('cuda')

print(torch.argmax(clf(img_tensor)))

tensor(2, device='cuda:0')
